In [6]:
# pip install ultralytics opencv-python numpy matplotlib

In [13]:
from ultralytics import YOLO
model = YOLO("yolo11x-seg.pt")  # or use yolov8n-seg.pt for faster inference


In [21]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model (detection)
model = YOLO("yolo11x.pt")  # or yolov8x.pt for better accuracy

# Input and output
input_video = "input.mp4"
output_video = "ball_only_output.mp4"

cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

print("Processing video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Start with green background
    green_frame = np.full_like(frame, (0, 255, 0))

    # Detect objects
    results = model(frame)[0]

    for box, cls in zip(results.boxes.xyxy, results.boxes.cls):
        if int(cls) == 32:  # 32 = sports ball in COCO
            x1, y1, x2, y2 = map(int, box)
            ball_crop = frame[y1:y2, x1:x2]

            # Preprocess for HoughCircles
            gray = cv2.cvtColor(ball_crop, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)

            circles = cv2.HoughCircles(
                gray,
                cv2.HOUGH_GRADIENT,
                dp=1.2,
                minDist=10,
                param1=50,
                param2=20,
                minRadius=3,
                maxRadius=(x2 - x1) // 2
            )

            if circles is not None:
                circles = np.uint16(np.around(circles))
                for cx, cy, r in circles[0, :1]:  # only the most confident
                    # Draw yellow ball on green background
                    cv2.circle(green_frame, (x1 + cx, y1 + cy), r, (0, 255, 255), -1)

    out.write(green_frame)

cap.release()
out.release()
print(f"✅ Done! Output saved to: {output_video}")


Processing video...

0: 640x384 2 persons, 1 sports ball, 10.7ms
Speed: 0.8ms preprocess, 10.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 sports ball, 10.0ms
Speed: 0.7ms preprocess, 10.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 10.5ms
Speed: 0.7ms preprocess, 10.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 10.5ms
Speed: 0.7ms preprocess, 10.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 10.4ms
Speed: 0.7ms preprocess, 10.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 10.4ms
Speed: 0.8ms preprocess, 10.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 10.4ms
Speed: 0.8ms preprocess, 10.4ms inference, 0.7ms postprocess per image at shape (1,

In [14]:
import cv2
import numpy as np

# Load input video
cap = cv2.VideoCapture("input.mp4")
width = int(cap.get(3))
height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video writer
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

In [15]:
def classify_team_color(player_roi):
    h, w, _ = player_roi.shape
    shirt_area = player_roi[:h // 2, :]  # Top half = shirt

    avg_color = shirt_area.mean(axis=0).mean(axis=0)  # BGR
    b, g, r = avg_color

    # Ball - Yellow
    if r > 180 and g > 180 and b < 100:
        return 'ball'

    # Goalie - Yellow jersey
    if r > 200 and g > 200 and b < 100:
        return 'goalie'

    # Team A - Blue shirt
    elif b > 150 and r < 120 and g < 130:
        return 'team_A'

    # Team B - White shirt
    elif r > 180 and g > 180 and b > 180:
        return 'team_B'

    else:
        return 'unknown'


In [17]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    mask_layer = np.full_like(frame, (0, 255, 0))  # Green background

    for seg, box in zip(results.masks.data, results.boxes.xyxy):
        seg = seg.cpu().numpy().astype(np.uint8)
        x1, y1, x2, y2 = map(int, box)
        roi = frame[y1:y2, x1:x2]
        label = classify_team_color(roi)

        if label == 'team_A':
            color = (255, 0, 0)       # Blue
        elif label == 'team_B':
            color = (255, 255, 255)   # White
        elif label == 'goalie':
            color = (0, 255, 255)     # Yellow
        elif label == 'ball':
            color = (0, 255, 255)     # Also Yellow (or you can change to black: (0, 0, 0))


        # Apply color mask
        seg_resized = cv2.resize(seg, (frame.shape[1], frame.shape[0]))
        mask_layer[seg_resized == 1] = color

    # Combine mask layer and write frame
    out.write(mask_layer)



0: 640x384 2 persons, 1 sports ball, 14.7ms
Speed: 8.9ms preprocess, 14.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 13.7ms
Speed: 0.8ms preprocess, 13.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 13.8ms
Speed: 0.7ms preprocess, 13.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 13.9ms
Speed: 0.7ms preprocess, 13.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 14.3ms
Speed: 0.7ms preprocess, 14.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 1 sports ball, 14.4ms
Speed: 0.7ms preprocess, 14.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 2 sports balls, 14.4ms
Speed: 0.7ms preprocess, 14.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 6

In [18]:
cap.release()
out.release()
cv2.destroyAllWindows()
